<a href="https://colab.research.google.com/github/ykitaguchi77/CongenitalGlaucoma_AI_project/blob/main/crop_corneal_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Extract corneal from congenital glaucoma dataset**

In [15]:
import glob
import codecs
import pandas as pd
import os
import cv2
import time
import csv
import numpy as np
import requests
import shutil

orig_image_path = glob.glob(f"{image_dir}/*")
dst_image_path = r"H:\先天性緑内障\dataset_extended"

orig_csv_path = r"H:\先天性緑内障\result_mobilenetv3.csv"

dst_csv_path = r"H:\先天性緑内障\cropped_eye_list.csv" 

# カスケードファイルのパス
#eye_cascade_path = "H:\haarcascade_eye.xml" #haarcascade_eye.xmlのあるパス


# カスケード分類器の特徴量取得
#eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

In [2]:
#Open csv
with codecs.open(orig_csv_path, "r", "Shift-JIS", "ignore") as file:
        df = pd.read_csv(file, index_col=None, header=0)
df

,pt_number,path,fold,label,pred,prob
0,1546,H:\先天性緑内障\dataset_250px\gla_ortho\1546_2.jpg,0,1,0,0.014977
1,1546,H:\先天性緑内障\dataset_250px\gla_ortho\1546_6.jpg,0,1,0,0.042091
2,1546,H:\先天性緑内障\dataset_250px\gla_ortho\1546_7.jpg,0,1,0,0.102872
3,1546,H:\先天性緑内障\dataset_250px\gla_ortho\1546_9.jpg,0,1,0,0.178373
4,1962,H:\先天性緑内障\dataset_250px\gla_ortho\1962_1.jpg,0,1,0,0.142860
...,...,...,...,...,...,...
915,9600,H:\先天性緑内障\dataset_250px\cont_exo\9600.jpg,4,0,0,0.008916
916,9622,H:\先天性緑内障\dataset_250px\cont_exo\9622.jpg,4,0,0,0.117579
917,9758,H:\先天性緑内障\dataset_250px\cont_exo\9758.jpg,4,0,0,0.008900
918,9810,H:\先天性緑内障\dataset_250px\cont_exo\9810.jpg,4,0,0,0.423352


In [3]:
def extract_patient_id(img_path: str)-> int:
    id = int(os.path.basename(img_path).split("_")[0].split("-")[0].split(".")[0])
    return id

df_gla = df[df["label"]==1]

# Extarct patients' ID
gla_id_list = [extract_patient_id(i) for i in df_gla["path"]]
gla_id_list = sorted(set(gla_id_list), key=int)
print(f"gla_pt_number: {len(gla_id_list)}")
print(f"gla_image_number: {len(df_gla)}")

#該当患者の画像リスト
gla_image_path_list = [i for i in orig_image_path if extract_patient_id(i) in gla_id_list]
print(f"gla_image_path_list: {len(gla_image_path_list)}")

gla_pt_number: 35
gla_image_number: 230
gla_image_path_list: 735


In [4]:
df_cont = df[df["label"]==0]

# Extarct patients' ID
cont_id_list = [extract_patient_id(i) for i in df_cont["path"]]
cont_id_list = sorted(set(cont_id_list), key=int)
print(f"cont_pt_number: {len(cont_id_list)}")
print(f"cont_image_number: {len(df_cont)}")

#該当患者の画像リスト(同じidあたり画像1枚とする)
cont_image_path_list = []
append_id_list = []
for path in orig_image_path:
    id = extract_patient_id(path)
    if id in cont_id_list:
        if id not in append_id_list:
          cont_image_path_list.append(path)
          append_id_list.append(id)


print(f"cont_image_path_list: {len(cont_image_path_list)}")

cont_pt_number: 674
cont_image_number: 690
cont_image_path_list: 674


In [ ]:
#処理するDirectoryの設定
directory = gla_image_path_list + cont_image_path_list
print(len(directory))

directory

In [ ]:
if os.path.exists(dst_image_path):
    shutil.rmtree(dst_image_path)
os.makedirs(dst_image_path)

for i, path in enumerate(directory):
    shutil.copy(path,f"{dst_image_path}\\{os.path.splitext(os.path.basename(path))[0]}.jpg" )
    print(i)

# **Crop periocular images**
LocalでHaarcascadeが使えないので、データをgdriveに載せて使う

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import csv
from random import randint
from time import sleep

import glob
import random
from PIL import Image
%matplotlib inline

#サポートパッチのインポート
from google.colab.patches import cv2_imshow
import cv2

plt.ion()   # interactive mode

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# unzip file
!cp "/content/drive/MyDrive/Deep_learning/Congenital_Glaucoma/glaucoma_extended_dataset.zip" .
!unzip glaucoma_extended_dataset.zip


In [5]:
#元画像フォルダ
in_path = '/content/dataset_extended'

#保存先フォルダ
out_path = '/content/dataset_periocular'

#CSVファイルのフォルダ
csv_path = '/content/glaucoma_perioucular_images_list.csv'

In [6]:
# カスケードファイルのパス
eye_cascade_path = '/content/drive/My Drive/Deep_learning/haarcascade_eye.xml'

# カスケード分類器の特徴量取得
eye_cascade = cv2.CascadeClassifier(eye_cascade_path)

In [22]:
#処理するDirectoryの設定
directory = glob.glob(f"{in_path}/*")
print(len(directory))

#ここにフォルダ番号を記載する (ex. [0:999])
processing_directory = directory[0:1409]
#print(processing_directory)
     

1409


In [20]:
processing_directory[0]

'/content/dataset_extended/4849-20191028-8-134035_07393886d2a571558d21930ab8ffa6ba6b7812c34bcb3d65cabfdf6188e11298.jpg'

In [23]:
#処理時間の計測
start = time.time()

with open(csv_path, 'a', newline='') as f:  #a:追記モード
        writer = csv.writer(f)
        fieldnames = ['Number', 'Folder', 'FileName']       
        writer.writerow(['Folder', 'Image_num', 'side (R:0, L:1)', 'ImageName', 'Number'])  #header
        
        k=0
        for i, path in enumerate(processing_directory):  #画像の分だけ
              print('')
              print(str(i))
              img = cv2.imread(path)
              img2 = img.copy()

              # 画像グレースケール化
              grayscale_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

              #300pix以上のもので目に見えるものを抽出
              eye_list = eye_cascade.detectMultiScale(grayscale_img, minSize=(300, 300))
              
              # 眼検出判定
              if len(eye_list) >= 1:
                  
                  print('目が' + str(len(eye_list)) +'個検出されました')
                  
                  pass
              else:
                  
                  print("eye detection error")
                  
                  break

              #画像の切り抜きと保存（連番にする）
              
              m=0
              for (ex, ey, ew, eh) in eye_list:
                  m+=1
                  #print("[ex,ey] = %d,%d [ew,eh] = %d,%d" %(ex, ey, ew, eh))
                  try:
                    cv2.rectangle(img2, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
                    img_cropped = img[int(ey-0.1*eh): int(ey+1.1*eh), int(ex-0.1*ew): int(ex+1.1*ew)]
                    img_resized = cv2.resize(img_cropped,(640,640)) #1辺640pixにresize 
                    #x, y = random.randint(0,27), random.randint(0,27) #ランダムな位置で224pixに切り取る
                    #img_randcrop = img_resized[y: y+223, x: x+223]
                  except:
                      pass

                  if ex <= 600:
                      side = 0 #600ピクセル目より左にあるのは右眼
                  else:
                      side = 1 #600ピクセル目より右にあるのは左眼
                  
                  '''
                  '''

                  #保存する際のよくわからないエラーをskipする
                  try:
                      cv2.imwrite(f'{dst_image_path}/{str(i)}-{side}.jpg', img_resized)
                      #対応表の作成
                      writer.writerow([str(i), side, str(extract_patient_id(path)), str(path)])

                  except:
                      pass

print('Process done!!')
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")


0

目が2個検出されました
1

目が2個検出されました
2

目が3個検出されました
3

目が2個検出されました
4

目が2個検出されました
5

目が1個検出されました
6

目が2個検出されました
7

目が1個検出されました
8

目が2個検出されました
9

目が1個検出されました
10

目が2個検出されました
11

目が2個検出されました
12

目が3個検出されました
13

eye detection error
Process done!!
elapsed_time:1.9599406719207764[sec]


In [12]:
import os
#assert os.path.isfile(face_cascade_path), 'haarcascade_frontalface_default.xml がない'
assert os.path.isfile(eye_cascade_path), 'eye_cascade_path.xml がない'